<b>Problem Statement</b>

This problem was done as a part of L&T FinHack 2 conducted by Analytics Vidya in January, 2020.

You have been appointed with the task of forecasting daily cases for next 3 months for 2 different business segments
aggregated at the country level keeping in consideration the following major Indian festivals (inclusive but not exhaustive
list): Diwali, Dussehra, Ganesh Chaturthi, Navratri, Holi etc. (You are free to use any publicly available open source external
datasets). Some other examples could be:

1. Weather 
2. Macroeconomic variables 

Note that the external dataset must belong to a reliable source.The train data has been provided in the following way:

For business segment 1, historical data has been made available at branch ID level 
For business segment 2, historical data has been made available at State level.

<b>Train File Variable Definition:</b>
1. application_date : Date of application segment 
2. segment : Business Segment (1/2) 
3. branch_id : Anonymised id for branch at which application was received 
4. state : State in which application was received (Karnataka, MP etc.) 
5. zone : Zone of state in which application was received (Central, East etc.) 
6. case_count (Target) : Number of cases/applications received

<b>Testing file Variable Definition:</b>
1. id : Unique id for each sample in test set 
2. application_date : Date of application 
3. segment : Business Segment (1/2)

Forecasting needs to be done at country level for the dates provided in test set for each segment.


### Import libraries

In [1]:
import pandas as pd
from datetime import datetime, time, date, timedelta
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

### Initializing variables with training and test file names

In [ ]:
train_file            = 'train_file.csv'
test_file             = 'test_file.csv'
output_predicted_file = 'test_predict_file.csv'

### Initializing dictionary with holidays for 4 years. 

In [3]:
#The dates were taken from the website - https://www.india-briefing.com/
holidays = { 2017: {'holi': '20170313' , 'muharram': '20171001', 'dussehra': '20170930', 'diwali': '20171019', 'christmas': '20171225'},
             2018: {'holi': '20180302' , 'muharram': '20180921', 'dussehra': '20181019', 'diwali': '20181107', 'christmas': '20181225'},
             2019: {'holi': '20190321' , 'muharram': '20190910', 'dussehra': '20191008', 'diwali': '20191027', 'christmas': '20191225'},
             2020: {'holi': '20200310' , 'muharram': '20200830', 'dussehra': '20201025', 'diwali': '20201114', 'christmas': '20201225'},
           }

### Various function definitions

In [4]:
def public_data_df():
    '''
    Function to create dataframe by merging data from 
    different public datasets.
    
    args:
    None
    
    except:
    None
    
    returns:
    cpi(pandas dataframe) : Pandas dataframe with Consumer Price Index(CPI) values
    sp(pandas dataframe)  : Pandas dataframe with Share Prices(SP) values
    bci(pandas dataframe) : Pandas dataframe with Business Confidence Index(BCI) values
    cli(pandas dataframe) : Pandas dataframe with Composite Lead Indicator(CLI) values
    ltir(pandas dataframe): Pandas dataframe with Long Term Interest Rate(LTIR) values
    wpi(pandas dataframe) : Pandas dataframe with Wholesale Price Index(WPI) values
    
    '''
    
    cpi  = pd.read_csv('raw_datasets/consumer_price_index.csv')
    sp   = pd.read_csv('raw_datasets/share_prices.csv')
    bci  = pd.read_csv('raw_datasets/business_confidence_index.csv')
    cli  = pd.read_csv('raw_datasets/composite_leading_indicator.csv')
    ltir = pd.read_csv('raw_datasets/long_term_interest_rates.csv')
    wpi  = pd.read_csv('raw_datasets/wholesale_price_index.csv')
    
    cpi = cpi[['Year', 'Month', 'Value']]
    cpi.columns = ['application_year', 'application_month','cpi']
    cpi['cpi-3m-mean']    = cpi['cpi'].rolling(window=3).mean()
    cpi['cpi-3m-std']     = cpi['cpi'].rolling(window=3).std()
    cpi['cpi-6m-mean']    = cpi['cpi'].rolling(window=6).mean()
    cpi['cpi-6m-std']     = cpi['cpi'].rolling(window=6).std()
    cpi['cpi-12m-mean']   = cpi['cpi'].rolling(window=12).mean()
    cpi['cpi-12m-std']    = cpi['cpi'].rolling(window=12).std()
    cpi['cpi-6m-excess']  = cpi['cpi-6m-mean'] / cpi['cpi-6m-std']
    cpi['cpi-12m-excess'] = cpi['cpi-12m-mean'] / cpi['cpi-12m-std']
    
    sp = sp[['Year', 'Month', 'Value']]
    sp.columns=['application_year', 'application_month','sp']
    sp['sp-3m-mean']    = sp['sp'].rolling(window=3).mean()
    sp['sp-3m-std']     = sp['sp'].rolling(window=3).std()
    sp['sp-6m-mean']    = sp['sp'].rolling(window=6).mean()
    sp['sp-6m-std']     = sp['sp'].rolling(window=6).std()
    sp['sp-12m-mean']   = sp['sp'].rolling(window=12).mean()
    sp['sp-12m-std']    = sp['sp'].rolling(window=12).std()
    sp['sp-6m-excess']  = sp['sp-6m-mean'] / sp['sp-6m-std']
    sp['sp-12m-excess'] = sp['sp-12m-mean'] / sp['sp-12m-std']
    
    bci = bci[['Year', 'Month', 'Value']]
    bci.columns=['application_year', 'application_month', 'bci']
    bci['bci-3m-mean']    = bci['bci'].rolling(window=3).mean()
    bci['bci-3m-std']     = bci['bci'].rolling(window=3).std()
    bci['bci-6m-mean']    = bci['bci'].rolling(window=6).mean()
    bci['bci-6m-std']     = bci['bci'].rolling(window=6).std()
    bci['bci-12m-mean']   = bci['bci'].rolling(window=12).mean()
    bci['bci-12m-std']    = bci['bci'].rolling(window=12).std()
    bci['bci-6m-excess']  = bci['bci-6m-mean'] / bci['bci-6m-std']
    bci['bci-12m-excess'] = bci['bci-12m-mean'] / bci['bci-12m-std']
    
    cli = cli[['Year', 'Month', 'Value']]
    cli.columns=['application_year', 'application_month','cli']
    cli['cli-3m-mean']    = cli['cli'].rolling(window=3).mean()
    cli['cli-3m-std']     = cli['cli'].rolling(window=3).std()
    cli['cli-6m-mean']    = cli['cli'].rolling(window=6).mean()
    cli['cli-6m-std']     = cli['cli'].rolling(window=6).std()
    cli['cli-12m-mean']   = cli['cli'].rolling(window=12).mean()
    cli['cli-12m-std']    = cli['cli'].rolling(window=12).std()
    cli['cli-6m-excess']  = cli['cli-6m-mean'] / cli['cli-6m-std']
    cli['cli-12m-excess'] = cli['cli-12m-mean'] / cli['cli-12m-std']
    
    ltir = ltir[['Year', 'Month', 'Value']]
    ltir.columns=['application_year', 'application_month','ltir']
    ltir['ltir-3m-mean']    = ltir['ltir'].rolling(window=3).mean()
    ltir['ltir-3m-std']     = ltir['ltir'].rolling(window=3).std()
    ltir['ltir-6m-mean']    = ltir['ltir'].rolling(window=6).mean()
    ltir['ltir-6m-std']     = ltir['ltir'].rolling(window=6).std()
    ltir['ltir-12m-mean']   = ltir['ltir'].rolling(window=12).mean()
    ltir['ltir-12m-std']    = ltir['ltir'].rolling(window=12).std()
    ltir['ltir-6m-excess']  = ltir['ltir-6m-mean'] / ltir['ltir-6m-std']
    ltir['ltir-12m-excess'] = ltir['ltir-12m-mean'] / ltir['ltir-12m-std']
    
    wpi = wpi[['Year', 'Month', 'Value']]
    wpi.columns=['application_year', 'application_month','wpi']
    wpi['wpi-3m-mean']    = wpi['wpi'].rolling(window=3).mean()
    wpi['wpi-3m-std']     = wpi['wpi'].rolling(window=3).std()
    wpi['wpi-6m-mean']    = wpi['wpi'].rolling(window=6).mean()
    wpi['wpi-6m-std']     = wpi['wpi'].rolling(window=6).std()
    wpi['wpi-12m-mean']   = wpi['wpi'].rolling(window=12).mean()
    wpi['wpi-12m-std']    = wpi['wpi'].rolling(window=12).std()
    wpi['wpi-6m-excess']  = wpi['wpi-6m-mean'] / wpi['wpi-6m-std']
    wpi['wpi-12m-excess'] = wpi['wpi-12m-mean'] / wpi['wpi-12m-std']
    
    return cpi.dropna(), sp.dropna(), bci.dropna(), cli.dropna(), ltir.dropna(), wpi.dropna()

In [5]:
def identify_qtr(x):
    '''
    Function to categories for quarter values
    
    args:
    x(int): Integer value for a quarter
    
    except:
    None
    
    returns:
    'QTR1'/'QTR2'/'QTR3'/'QTR4'(str): Category for a 
                                      quarter
    '''
    
    if x >= 1 and x <= 3:
        return 'QTR1'
    elif x >= 4 and x <= 6:
        return 'QTR2'
    elif x >= 7 and x <= 9:
        return 'QTR3'
    elif x >= 10 and x <= 12:
        return 'QTR4'
    else:
        return 'Invalid QTR'

In [6]:
def identify_daytype(x):
    '''
    Function to identify day category
    given a particular day number for a 
    week
    
    args:
    day number(int): Integer value for day of week
    
    except:
    None
    
    returns:
    'weekday'/'weekend'/'Invalid day type': Category type
                             representing a day
    
    '''
    
    if x >= 0 and x <= 5:
        return 'weekday'
    elif x == 6 or x == 7:
        return 'weekend'
    else:
        return 'Invalid day type'

In [7]:
def identify_season(x):
    '''
    Function to identify season given a month
    
    args:
    x(int): Integer representing a month
    
    except:
    None
    
    returns:
    'summer'/'monsoon'/'autumn'/'winter'/'Invalid season': Name of season
    
    '''
    
    if x >= 3 and x <= 5:
        return 'summer'
    elif x >= 6 and x <= 9:
        return 'monsoon'
    elif x == 10 or x == 11:
        return 'autumn'
    elif x == 12 or x == 1 or x == 2:
        return 'winter'
    else:
        return 'Invalid season'

In [8]:
def dist_from_holiday(x, festival):
    '''
    Function to count distance in terms of no. of days 
    from a festival date
    
    args:
    x(date): Day in consideration
    festival: Festival in consideration
    
    except:
    None
    
    returns:
    distance(datetime.days): Difference between date in consideration
                             and festival date
    
    '''
    dt_year = x.year
    dt_hol = datetime.strptime(holidays[dt_year][festival], '%Y%m%d')
    if (dt_hol - x).days < 0:
        dt_year = dt_year + 1 
        dt_hol = datetime.strptime(holidays[dt_year][festival], '%Y%m%d')
        return (dt_hol - x).days
    else:
        return (dt_hol - x).days  

In [9]:
def merge_public_data(df):
    '''
    Function to merge public datasets with the training
    or testing data
    
    args:
    df(pandas dataframe): Pandas dataframe with training 
                          or testing data
    
    except:
    None
    
    returns:
    df(pandas dataframe): Pandas dataframe after merging 
                          training or testing data with
                          public data
    
    '''
    cpi, sp, bci, cli, ltir, wpi = public_data_df()
    pd_df = pd.merge(cpi, sp, on=['application_year', 'application_month']).\
            merge(bci, on=['application_year', 'application_month']).\
            merge(cli, on=['application_year', 'application_month']).\
            merge(ltir, on=['application_year', 'application_month']).\
            merge(wpi, on=['application_year', 'application_month'])
    df = pd.merge(df, pd_df, on=['application_year', 'application_month'], how='left')         
    
    return df

In [10]:
def prepare_training_frame(data_file):
    '''
    Function to prepare training dataframe
    
    args:
    data_file(str): Name of file with training data
    
    except:
    None
    
    returns:
    df(pandas dataframe): Pandas dataframe with features
                          generated from training data
    
    '''
    
    df = pd.read_csv(data_file)
    df = df[['application_date', 'segment', 'case_count']]
    df = df.groupby(['application_date', 'segment']).agg({'case_count':['sum']}).reset_index()
    df.columns = ['application_date', 'segment', 'case_count']
    df['application_date']    = pd.to_datetime(df['application_date'], format='%Y-%m-%d')
    df['application_year']    = df['application_date'].dt.year
    df['application_month']   = df['application_date'].dt.month
    df['application_dow']     = df['application_date'].dt.dayofweek
    df['application_qtr']     = df['application_month'].apply(lambda x: identify_qtr(x))
    df['application_daytype'] = df['application_dow'].apply(lambda x: identify_daytype(x))
    df['application_season']  = df['application_month'].apply(lambda x: identify_season(x))
    df['dist_holi']           = df['application_date'].apply(lambda x: dist_from_holiday(x, 'holi'))
    df['dist_muharram']       = df['application_date'].apply(lambda x: dist_from_holiday(x, 'muharram'))
    df['dist_dussehra']       = df['application_date'].apply(lambda x: dist_from_holiday(x, 'dussehra'))
    df['dist_diwali']         = df['application_date'].apply(lambda x: dist_from_holiday(x, 'diwali'))
    df['dist_christmas']      = df['application_date'].apply(lambda x: dist_from_holiday(x, 'christmas'))
    df = merge_public_data(df)
    
    return df.fillna(0)

In [43]:
def prepare_testing_frame(data_file):
    '''
    Function to prepare testing dataframe
    
    args:
    data_file(str): Name of file with testing data
    
    except:
    None
    
    returns:
    df(pandas dataframe): Pandas dataframe with features
                          generated from testing data
                          
    '''
    
    df = pd.read_csv(data_file)
    df.columns = ['id', 'application_date', 'segment']
    df.set_index('id')
    df['application_date']    = pd.to_datetime(df['application_date'], format='%Y-%m-%d')
    df['application_year']    = df['application_date'].dt.year
    df['application_month']   = df['application_date'].dt.month
    df['application_dow']     = df['application_date'].dt.dayofweek
    df['application_qtr']     = df['application_month'].apply(lambda x: identify_qtr(x))
    df['application_daytype'] = df['application_dow'].apply(lambda x: identify_daytype(x))
    df['application_season']  = df['application_month'].apply(lambda x: identify_season(x))
    df['dist_holi']           = df['application_date'].apply(lambda x: dist_from_holiday(x, 'holi'))
    df['dist_muharram']       = df['application_date'].apply(lambda x: dist_from_holiday(x, 'muharram'))
    df['dist_dussehra']       = df['application_date'].apply(lambda x: dist_from_holiday(x, 'dussehra'))
    df['dist_diwali']         = df['application_date'].apply(lambda x: dist_from_holiday(x, 'diwali'))
    df['dist_christmas']      = df['application_date'].apply(lambda x: dist_from_holiday(x, 'christmas'))
    df = merge_public_data(df)
        
    return df.fillna(0)

### Generate simple linear model and plot characteristics

In [158]:
train_df = prepare_training_frame(train_file)
cols = ['segment', 'application_qtr', 'application_daytype', 'application_season', 'dist_holi','dist_muharram','dist_dussehra','dist_diwali','dist_christmas', 'wpi', 'wpi-3m-mean', 'wpi-6m-mean', 'case_count']
train_df = train_df[cols]
train_df['segment'] = train_df['segment'].astype('category')
train_df['application_qtr'] = train_df['application_qtr'].astype('category')
train_df['application_daytype'] = train_df['application_daytype'].astype('category')
train_df['application_season'] = train_df['application_season'].astype('category')
train_df = pd.get_dummies(train_df)

tr_df_X = train_df.drop(['case_count'], axis=1).iloc[:-500]
tr_df_Y = train_df['case_count'].iloc[:-500]
te_df_X = train_df.drop(['case_count'], axis=1).iloc[-500:]
te_df_Y = train_df['case_count'].iloc[-500:]
regr = linear_model.LinearRegression()
regr.fit(tr_df_X, tr_df_Y)
te_pred_df_Y = regr.predict(te_df_X)
plt.scatter(te_df_Y, te_pred_df_Y)
plt.show()
print("Mean Squared Error : {}".format(mean_squared_error(te_df_Y, te_pred_df_Y)))
print("R squared : {}".format(r2_score(te_df_Y, te_pred_df_Y)))

### Predict case_counts using testing data and write to file

In [160]:
test_df = prepare_testing_frame(test_file)
cols = ['segment', 'application_qtr', 'application_daytype', 'application_season', 'dist_holi','dist_muharram','dist_dussehra','dist_diwali','dist_christmas', 'wpi', 'wpi-3m-mean', 'wpi-6m-mean']
filt_test_df = test_df.loc[:, cols]

filt_test_df['segment'] = filt_test_df['segment'].astype('category')
filt_test_df['application_qtr'] = filt_test_df['application_qtr'].astype('category')
filt_test_df['application_daytype'] = filt_test_df['application_daytype'].astype('category')
filt_test_df['application_season'] = filt_test_df['application_season'].astype('category')
filt_test_df = pd.get_dummies(filt_test_df)

filt_test_df['application_qtr_QTR1'] = [0] * len(filt_test_df)
filt_test_df['application_qtr_QTR2'] = [0] * len(filt_test_df)
filt_test_df['application_season_summer'] = [0] * len(filt_test_df)
filt_test_df['application_season_winter'] = [0] * len(filt_test_df)

filt_test_df = filt_test_df[tr_df_X.columns]
filt_test_pred = regr.predict(filt_test_df)
test_df['case_count'] = filt_test_pred
test_df['case_count'] = test_df['case_count'].round(2)
test_df[['id', 'application_date', 'segment', 'case_count']].to_csv(test_predict_file, index=False)